In [ ]:
#Installing packages
!pip install geoio
!pip install geopandas

In [ ]:
#Importing necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import geoio
from google.colab import drive
import sys
from shapely.geometry import Polygon

In [ ]:
#Mounting Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Downloading nightlights satellite data from SNPP VIIRS
!wget https://data.ngdc.noaa.gov/instruments/remote-sensing/passive/spectrometers-radiometers/imaging/viirs/dnb_composites/v10/2015/SVDNB_npp_20150101-20151231_75N060E_v10_c201701311200.tgz

--2021-01-05 04:56:27--  https://data.ngdc.noaa.gov/instruments/remote-sensing/passive/spectrometers-radiometers/imaging/viirs/dnb_composites/v10/2015/SVDNB_npp_20150101-20151231_75N060E_v10_c201701311200.tgz
Resolving data.ngdc.noaa.gov (data.ngdc.noaa.gov)... 140.172.190.8
Connecting to data.ngdc.noaa.gov (data.ngdc.noaa.gov)|140.172.190.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4147680700 (3.9G) [application/x-gzip]
Saving to: ‘SVDNB_npp_20150101-20151231_75N060E_v10_c201701311200.tgz’

SVDNB_npp_20150101- 100%[===================>]   3.86G  24.8MB/s    in 3m 7s   

2021-01-05 04:59:34 (21.2 MB/s) - ‘SVDNB_npp_20150101-20151231_75N060E_v10_c201701311200.tgz’ saved [4147680700/4147680700]



In [ ]:
#Uncompressing the downloaded file
!tar -xvf  'SVDNB_npp_20150101-20151231_75N060E_v10_c201701311200.tgz' -C '/content/gdrive/My Drive/data/75N060E'

README_dnb_composites_v1.txt
SVDNB_npp_20150101-20151231_75N060E_vcm-ntl_v10_c201701311200.avg_rade9.tif
SVDNB_npp_20150101-20151231_75N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif
SVDNB_npp_20150101-20151231_75N060E_vcm-orm_v10_c201701311200.avg_rade9.tif
SVDNB_npp_20150101-20151231_75N060E_vcm_v10_c201701311200.avg_rade9.tif
SVDNB_npp_20150101-20151231_75N060E_vcm_v10_c201701311200.cf_cvg.tif
SVDNB_npp_20150101-20151231_75N060E_vcm_v10_c201701311200.cvg.tif


In [ ]:
#Defining base directory
BASE_DIR = '/content/gdrive/My Drive/data/'
NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, '/content/gdrive/My Drive/data/75N060E/SVDNB_npp_20150101-20151231_75N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif')]

In [ ]:
sys.path.append(BASE_DIR)

In [ ]:
#Loading the 1km hex files for Delhi
delhi_hex = gpd.read_file('delhi_hex_1km.shp')
delhi_hex.head()

,id,left,top,right,bottom,geometry
0,201.0,7.398443e+06,3.521635e+06,7.399598e+06,3.520635e+06,"POLYGON ((76.83489 28.59082, 76.83503 28.59140..."
1,202.0,7.398443e+06,3.520635e+06,7.399598e+06,3.519635e+06,"POLYGON ((76.83302 28.58196, 76.83292 28.58251..."
2,203.0,7.398443e+06,3.519635e+06,7.399598e+06,3.518635e+06,"POLYGON ((76.83465 28.57309, 76.83302 28.58196..."
3,204.0,7.398443e+06,3.518635e+06,7.399598e+06,3.517635e+06,"POLYGON ((76.83604 28.56552, 76.83465 28.57309..."
4,205.0,7.398443e+06,3.517635e+06,7.399598e+06,3.516635e+06,"POLYGON ((76.83733 28.55855, 76.83670 28.56195..."


In [ ]:
#Getting longitude and latitude boundaries for each hex along with its centroid coordinates
vector_bound_coordinates= delhi_hex['geometry']
delhi_hex[['min_lon','min_lat','max_lon','max_lat']] = vector_bound_coordinates.bounds
delhi_hex["cluster_lon"] = delhi_hex.centroid.map(lambda p: p.x)
delhi_hex["cluster_lat"] = delhi_hex.centroid.map(lambda p: p.y)
delhi_hex.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




,id,left,top,right,bottom,geometry,min_lon,min_lat,max_lon,max_lat,cluster_lon,cluster_lat
0,201.0,7.398443e+06,3.521635e+06,7.399598e+06,3.520635e+06,"POLYGON ((76.83489 28.59082, 76.83503 28.59140...",76.834893,28.590825,76.835526,28.591400,76.835165,28.591131
1,202.0,7.398443e+06,3.520635e+06,7.399598e+06,3.519635e+06,"POLYGON ((76.83302 28.58196, 76.83292 28.58251...",76.832916,28.581956,76.838164,28.590825,76.835325,28.585848
2,203.0,7.398443e+06,3.519635e+06,7.399598e+06,3.518635e+06,"POLYGON ((76.83465 28.57309, 76.83302 28.58196...",76.833018,28.573087,76.838164,28.581956,76.835343,28.577948
3,204.0,7.398443e+06,3.518635e+06,7.399598e+06,3.517635e+06,"POLYGON ((76.83604 28.56552, 76.83465 28.57309...",76.834651,28.565519,76.838164,28.573087,76.836257,28.569340
4,205.0,7.398443e+06,3.517635e+06,7.399598e+06,3.516635e+06,"POLYGON ((76.83733 28.55855, 76.83670 28.56195...",76.836703,28.558551,76.838164,28.561946,76.837398,28.560094


In [ ]:
#Getting downloaded GeoTIFFs
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [ ]:
#Function for calculating nightlights
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    
    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = r.min_lat,r.min_lon,r.max_lat,r.max_lon
        print(r.cluster_lat,r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        print(xminPixel,ymaxPixel)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.min_lat}, {r.min_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.min_lat}, {r.min_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [ ]:
#Getting the TIFF array
tif_array = np.squeeze(tifs[0].get_data())

In [ ]:
#Adding nightlights to each hex 
add_nightlights(delhi_hex, tifs[0], tif_array)

28.591130546903344 76.83516478253736
4040.8742604147105 11138.70193932097
28.585848418917024 76.83532502356834
4040.3998700245684 11140.830563701911
28.57794781358727 76.83534269166061
4040.4242868131205 11142.959008579724
28.569340361032665 76.83625707503644
4040.816251890098 11144.77525072724
28.560094015077944 76.83739838149388
4041.3087172915966 11146.44776707948
28.588897684771084 76.84088638248012
4040.9397910441166 11139.766273955358
28.581955696334315 76.84116119351783
4040.9397910441166 11141.894808572684
28.57308717589452 76.84116119351783
4040.9397910441166 11144.02316373508
28.564227744774733 76.84129460181177
4041.150749209651 11146.151339538905
28.557051045982824 76.84238269578368
4041.4587978238897 11147.747520680945
28.591181691800834 76.85290156326838
4044.902620030898 11138.70193932097
28.58626423046689 76.85018379988186
4043.098085641819 11140.830563701911
28.57752134264831 76.85015408774687
4043.098085641819 11142.959008579724
28.568653196022762 76.85015408774687
40

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in true_divide



28.522456461498127 76.98608426890276
4075.903143867562 11155.799643525988
28.834536063856532 76.99390668014604
4077.6307992050624 11081.160875068714
28.82613449975583 76.99404039541149
4077.6307992050624 11083.294382680386
28.817245264074202 76.99404039541147
4077.6307992050624 11085.427708164481
28.80835678705338 76.99404039541147
4077.6307992050624 11087.560851618968
28.79946906828529 76.99404039541146
4077.6307992050624 11089.693813141756
28.790582107362113 76.99404039541147
4077.6307992050624 11091.826592830686
28.78169590387627 76.99404039541149
4077.6307992050624 11093.959190783555
28.772810457420416 76.99404039541147
4077.6307992050624 11096.091607098086
28.763925767587487 76.99404039541146
4077.6307992050624 11098.223841871953
28.755041833970648 76.99404039541147
4077.6307992050624 11100.35589520277
28.746158656163285 76.99404039541149
4077.6307992050624 11102.487767188084
28.737276233759058 76.99404039541147
4077.6307992050624 11104.619457925395
28.728394566351877 76.994040395

In [ ]:
del tif_array
import gc
gc.collect()

0

In [ ]:
import psutil
psutil.virtual_memory()

svmem(total=13653602304, available=11987304448, percent=12.2, used=5545418752, free=193929216, active=1520988160, inactive=11528523776, buffers=80228352, cached=7834025984, shared=958464, slab=295165952)

In [ ]:
#Check
delhi_hex.head()

,id,left,top,right,bottom,geometry,min_lon,min_lat,max_lon,max_lat,cluster_lon,cluster_lat,nightlights
0,201.0,7.398443e+06,3.521635e+06,7.399598e+06,3.520635e+06,"POLYGON ((76.83489 28.59082, 76.83503 28.59140...",76.834893,28.590825,76.835526,28.591400,76.835165,28.591131,NaN
1,202.0,7.398443e+06,3.520635e+06,7.399598e+06,3.519635e+06,"POLYGON ((76.83302 28.58196, 76.83292 28.58251...",76.832916,28.581956,76.838164,28.590825,76.835325,28.585848,1.258015
2,203.0,7.398443e+06,3.519635e+06,7.399598e+06,3.518635e+06,"POLYGON ((76.83465 28.57309, 76.83302 28.58196...",76.833018,28.573087,76.838164,28.581956,76.835343,28.577948,0.000000
3,204.0,7.398443e+06,3.518635e+06,7.399598e+06,3.517635e+06,"POLYGON ((76.83604 28.56552, 76.83465 28.57309...",76.834651,28.565519,76.838164,28.573087,76.836257,28.569340,1.482027
4,205.0,7.398443e+06,3.517635e+06,7.399598e+06,3.516635e+06,"POLYGON ((76.83733 28.55855, 76.83670 28.56195...",76.836703,28.558551,76.838164,28.561946,76.837398,28.560094,NaN


In [ ]:
#Saving files
delhi_hex.to_file("delhi_hex_1km.json", driver="GeoJSON")